# <a id='toc1_'></a>[Data Prep and Cleanup](#toc0_)

TODO: add a general note about what our data entails, including a brief discussion of the timespan were are using.

The goal of this notebook is:
- load data from datasets made available by different sources 
- perform basic clean-up tasks, including filtering overlapping timeframe
- add basic new features
- standardize feature names and create codebook
- generate base dataset for the next steps of the project

**Table of contents**<a id='toc0_'></a>    
- [Data Prep and Cleanup](#toc1_)    
  - [Census Data](#toc1_1_)    
    - [TODO: Demographics](#toc1_1_1_)    
    - [Housing, 2021](#toc1_1_2_)    
  - [Insurance Data](#toc1_2_)    
    - [Renewals](#toc1_2_1_)    
    - [Premiums, Claims, and Losses](#toc1_2_2_)    
    - [FAIR Plan (2022)](#toc1_2_3_)    
  - [TODO: Zillow Data](#toc1_3_)    
  - [TODO: Disaster Data](#toc1_4_)    
- [OLD CODE](#toc2_)    
  - [FAIR Plan 2 (2020-2024)](#toc2_1_)    
  - [FEMA Projected Premium Increases (2021, 2025)](#toc2_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# base folders
RAW_DATA_DIR = Path('../raw_data/')
DATA_DIR = Path('../data/')
CLEAN_DATA_DIR = Path('../clean_data/')

In [ ]:
# years to slice the data
start_year = 2018
end_year = 2021

## <a id='toc1_1_'></a>[Census Data](#toc0_)


TODO: add description and include Allison's data

### <a id='toc1_1_1_'></a>[TODO: Demographic](#toc0_)

### <a id='toc1_1_2_'></a>[Housing (2021)](#toc0_)

TODO: add description

TODO: how to deal with that

IMPORTANT: median_home_value is capped at $2,000,001

In [4]:
# number of housing units
housing_units = pd.read_csv(RAW_DATA_DIR / 'Housing Units in Census Zip Code Tabulation Areas of California (2021).csv')
housing_units = housing_units[['Entity properties name', 'Variable observation value']]
housing_units.columns=['ZIP Code', 'Housing Units']
housing_units.set_index('ZIP Code', inplace=True)

# median gross rent
gross_rent = pd.read_csv(RAW_DATA_DIR / 'Median Gross Rent of Housing Unit_ With Cash Rent in Census Zip Code Tabulation Areas of California (2021).csv')
gross_rent = gross_rent[['Entity properties name', 'Variable observation value']]
gross_rent.columns=['ZIP Code', 'Median Gross Rent ($)']
gross_rent.set_index('ZIP Code', inplace=True)

# median ownership costs
# Note: This dataset contains the median cost of housing units without mortgage
owner_cost = pd.read_csv(RAW_DATA_DIR / 'Median Cost of Housing Unit (Selected Monthly Owner Costs)_ Without Mortgage in Census Zip Code Tabulation Areas of California (2021).csv')
owner_cost = owner_cost[['Entity properties name', 'Variable observation value']]
owner_cost.columns=['ZIP Code', 'Median Owner Cost ($)']
owner_cost.set_index('ZIP Code', inplace=True)

# median home value
home_value = pd.read_csv(RAW_DATA_DIR / 'Median Home Value of Housing Unit_ Occupied Housing Unit, Owner Occupied in Census Zip Code Tabulation Areas of California (2021).csv')
home_value = home_value[['Entity properties name', 'Variable observation value']]
home_value.columns=['ZIP Code', 'Median Home Value - Census ($)']
home_value.set_index('ZIP Code', inplace=True)

housing = pd.concat([housing_units, gross_rent, owner_cost, home_value], axis=1).dropna()
housing.index = housing.index.astype(str)


In [5]:
housing.sample(3)

,Housing Units,Median Gross Rent ($),Median Owner Cost ($),Median Home Value - Census ($)
ZIP Code,,,,
95448,8296,1963.0,824.0,919400.0
93040,591,1417.0,555.0,393500.0
94973,708,1878.0,537.0,957400.0


In [ ]:
# readme = """
# Dataset: housing21.csv
# Coverage: 2022

# 'ZIP Code'
# 'housing_units' - number of housing units in the ZIP code area
# 'median_gross_rent' - median gross rent of housing units with cash rent in the ZIP code area
# 'median_ownership_cost' - median cost of housing units without mortgage in the ZIP code area
# 'median_home_value' - median home value of occupied housing units, owner occupied in the ZIP code area
# """

# with open(DATA_DIR / '2018_2021' / 'housing21.txt', 'w') as file:
#     file.write(readme)

## <a id='toc1_2_'></a>[Insurance Data](#toc0_)

California requires insurers with written premiums above of $10 million to submit a biennial report to the Insurance Commissioner with its residential property experience data for the previous two years. The data is processed by the Department of Insurance and the aggregates are published at zipcode level, including information about the number of policies, renewals, premiums, and losses. In this project, we used the following datasets:

- [New, renewed, and non-renewed insurance policies, 2015-2021](https://www.insurance.ca.gov/01-consumers/200-wrr/upload/Residential-Insurance-Policy-Analysis-by-County-2015-to-2021-2.pdf) 
- [Earned premiums, claims, and losses in residential units, 2018-2023](https://www.insurance.ca.gov/01-consumers/200-wrr/WildfireRiskInfoRpt.cfm)

Besides "regular" insurance data, we also include information about California's FAIR Plan. The California FAIR Plan provides basic insurance coverage for high-risk properties when traditional insurance companies will not. It has recently expanded to offer higher coverage limits of $3 million for residential policyholders and $20 million for commercial policies per location, serving as a safety net for properties that can't obtain coverage in the standard insurance market. The available data comes from these two datasets:
- [Residential Structures Insured under a FAIR Plan Policy, 2022](https://www.insurance.ca.gov/01-consumers/200-wrr/upload/Number-of-Residential-Dwelling-Units-Insured-in-2022-FAIR-Plan-vs-Voluntary.pdf)
- [Residential policies  by the program, 2020-2024](https://www.cfpnet.com/wp-content/uploads/2024/11/CFP5yearPIFGrowthbyzipcodethrough09302024(Residential%20line)20241112v001.pdf)
- [Residential exposure covered by the program, 2020-2024](https://www.cfpnet.com/wp-content/uploads/2024/11/CFP5yearTIVGrowthbyzipcodethrough09302024(Residentialline)20241112v001.pdf)

*** IMPORTANT NOTE: the best available data covers only 2022, and will be used a the target for our model *** 

### <a id='toc1_2_1_'></a>[Renewals](#toc0_)

In [64]:
# loading and performing initial processing on the renewals data
renewals = pd.read_excel(RAW_DATA_DIR / 'Residential-Property-Voluntary-Market-New-Renew-NonRenew-by-ZIP-2015-2021.xlsx', dtype={'ZIP Code': str})

# removing zipcodes not associated with a county
renewals = renewals[renewals['County'].isnull() == False]

# keeping only columns of interest and renaming
cols = ["ZIP Code", "Year", "New", "Renewed", "Insured-Initiated Nonrenewed", "Insurer-Initiated Nonrenewed"]
renewals = renewals[cols].copy()

renaming = {
    'New' : 'New Policies',
    'Renewed': 'Renewed Policies',
    'Insured-Initiated Nonrenewed': 'Nonrenewed Policies (by Owner)',
    'Insurer-Initiated Nonrenewed': 'Nonrenewed Policies (by Company)',
}

renewals.rename(columns=renaming, inplace=True)

renewals.sample(3)

,ZIP Code,Year,New Policies,Renewed Policies,Nonrenewed Policies (by Owner),Nonrenewed Policies (by Company)
12752,95073,2020,388,3104,275,127
13611,91107,2021,945,8569,800,170
1259,93560,2015,774,4587,531,133


The original dataset provides raw counts of policy renewals and includes multiple years. Based on this information, we can calculate some extra features, including the relative importance of each count (i.e., percentages) and their change over time.

In [65]:
# number of non-renewed policies and  expiring policies (or contracts up to renewal) 
renewals['Nonrenewed Policies'] = renewals['Nonrenewed Policies (by Owner)'] + renewals['Nonrenewed Policies (by Company)']
renewals['Expiring Policies'] = renewals['Nonrenewed Policies'] + renewals['Renewed Policies']

In [66]:
# filtering years of interest
cond1 = renewals['Year'] >= start_year
cond2 = renewals['Year'] <= end_year

renewals_filtered = renewals.loc[cond1 & cond2].copy()
renewals_filtered = renewals_filtered.groupby('ZIP Code', as_index=False).sum()
renewals_filtered.drop(columns='Year', inplace=True)

renewals_filtered.sample(3)

,ZIP Code,New Policies,Renewed Policies,Nonrenewed Policies (by Owner),Nonrenewed Policies (by Company),Nonrenewed Policies,Expiring Policies
1089,93618,2943,20823,1841,699,2540,23363
1150,93720,6489,48526,5759,744,6503,55029
198,91010,2094,19829,1569,406,1975,21804


In [67]:
# percentage of non-renewed policies of the expiring policies
renewals_filtered['% Nonrenewed Policies'] = renewals_filtered['Nonrenewed Policies'] / renewals_filtered['Expiring Policies']

# percentage of policies not-renewals by the initiative of the owner or company
renewals_filtered['% Nonrenewed Policies (by Owner)'] = renewals_filtered['Nonrenewed Policies (by Owner)'] / renewals_filtered['Expiring Policies']
renewals_filtered['% Nonrenewed Policies (by Company)'] = renewals_filtered['Nonrenewed Policies (by Company)'] / renewals_filtered['Expiring Policies']

renewals_filtered.sample(3)

,ZIP Code,New Policies,Renewed Policies,Nonrenewed Policies (by Owner),Nonrenewed Policies (by Company),Nonrenewed Policies,Expiring Policies,% Nonrenewed Policies,% Nonrenewed Policies (by Owner),% Nonrenewed Policies (by Company)
290,91410,0,3,1,0,1,4,0.250000,0.250000,0.000000
1257,94117,1305,14715,1106,244,1350,16065,0.084034,0.068845,0.015188
1801,95554,53,415,41,31,72,487,0.147844,0.084189,0.063655


In [68]:
# # TODO: move this one to feature engineering section

# # ratio of new policies to non-renewed policies
# renewals_filtered['ratio_new_to_nonrenewed'] = renewals_filtered['new_policies'] / (renewals_filtered['owner_nonrenewed'] + renewals_filtered['company_nonrenewed'])

In [69]:
# calculating change over time based on the start and end years
cond1 = renewals['Year'] == start_year
cond2 = renewals['Year'] == end_year

renewals_change = renewals[cond1 | cond2].copy().sort_values(['ZIP Code', 'Year']).set_index('ZIP Code')

renewals_change = renewals_change.groupby(['ZIP Code']).pct_change().dropna().copy().drop(columns='Year')
renewals_change.replace([np.inf, -np.inf], np.nan, inplace=True)

renewals_change.columns = ['% Change - ' + col for col in renewals_change.columns]
renewals_change.reset_index(inplace=True)
renewals_change.sample(3)

,ZIP Code,% Change - New Policies,% Change - Renewed Policies,% Change - Nonrenewed Policies (by Owner),% Change - Nonrenewed Policies (by Company),% Change - Nonrenewed Policies,% Change - Expiring Policies
1305,95427,0.000000,0.072000,-0.200000,0.400000,0.100000,0.074074
1418,95627,0.045455,0.011962,0.314286,0.038462,0.239583,0.035408
698,93265,0.186528,0.016578,0.333333,1.750000,0.727273,0.094985


In [70]:
# merging datasets 
renewals_change['ZIP Code'] = renewals_change['ZIP Code'].astype(str)
renewals_filtered['ZIP Code'] = renewals_filtered['ZIP Code'].astype(str)

renewals_reworked = pd.merge(renewals_filtered, renewals_change, on='ZIP Code')
renewals_reworked.set_index('ZIP Code', inplace=True)
renewals_reworked.sample(3)

,New Policies,Renewed Policies,Nonrenewed Policies (by Owner),Nonrenewed Policies (by Company),Nonrenewed Policies,Expiring Policies,% Nonrenewed Policies,% Nonrenewed Policies (by Owner),% Nonrenewed Policies (by Company),% Change - New Policies,% Change - Renewed Policies,% Change - Nonrenewed Policies (by Owner),% Change - Nonrenewed Policies (by Company),% Change - Nonrenewed Policies,% Change - Expiring Policies
ZIP Code,,,,,,,,,,,,,,,
93022,953,7086,597,345,942,8028,0.117339,0.074365,0.042975,0.585106,-0.050138,0.257576,1.574074,0.639785,0.013993
95315,1211,9708,884,225,1109,10817,0.102524,0.081723,0.020801,0.056478,0.031720,0.074419,0.000000,0.057554,0.034405
92225,2224,16230,1549,510,2059,18289,0.112581,0.084696,0.027886,0.233333,0.010665,0.356522,-0.029197,0.246888,0.035888


In [74]:
renewals_reworked.info(), renewals_reworked.index

<class 'pandas.core.frame.DataFrame'>
Index: 1713 entries, 90001 to 96162
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   New Policies                                 1713 non-null   int64  
 1   Renewed Policies                             1713 non-null   int64  
 2   Nonrenewed Policies (by Owner)               1713 non-null   int64  
 3   Nonrenewed Policies (by Company)             1713 non-null   int64  
 4   Nonrenewed Policies                          1713 non-null   int64  
 5   Expiring Policies                            1713 non-null   int64  
 6   % Nonrenewed Policies                        1713 non-null   float64
 7   % Nonrenewed Policies (by Owner)             1713 non-null   float64
 8   % Nonrenewed Policies (by Company)           1713 non-null   float64
 9   % Change - New Policies                      1699 non-null   float64
 10  

(None,
 Index(['90001', '90002', '90003', '90004', '90005', '90006', '90007', '90008',
        '90010', '90011',
        ...
        '96142', '96143', '96145', '96146', '96148', '96150', '96151', '96160',
        '96161', '96162'],
       dtype='object', name='ZIP Code', length=1713))

In [ ]:
# #renewals.to_csv(DATA_DIR / 'renewals_yearly.csv')
# renewals_reworked.to_csv(DATA_DIR / '2018_2021' / 'policy_renewals.csv')

# readme = """
# Dataset: policy_renewals.csv
# Coverage: 2018-2021


# 'ZIP Code'
# 'new_policies': number of new policies underwritten
# 'renewed_policies': number of policies renewed
# 'company_nonrenewed': number of policies not renewed by insurance company initiative
# 'owner_nonrenewed': number of policies not renewed by homeowner initiative
# 'pct_nonrenewed_policies': percentage of non-renewed policies from expiring policies 
# 'pct_nonrenewed_owner': percentage of non-renewed policies initiated by homeowners
# 'pct_nonrenewed_company': percentage of non-renewed policies initiated by insurance companies
# 'ratio_new_to_nonrenewed': ratio between non-renewed policies and new policies
# 'change_new_policies': percentual growth of new policies
# 'change_renewed_policies': percentual growth of renewed policies
# 'change_nonrenewed_owner': percentual growth of non-renewed policies initiated by homeowners
# 'change_nonrenewed_company': percentual growth of non-renewed policies initiated by insurance companies
# """

# with open(DATA_DIR / '2018_2021' / 'policy_renewals.txt', 'w') as file:
#     file.write(readme)

### <a id='toc1_2_2_'></a>[Premiums, Claims, and Losses](#toc0_)

This dataset covers certain types of residential policies (Dwelling Fire policies, Homeowners policies, Earthquake policies) and include information about total earned premiums as well as number of claims and total losses paid by insurance companies for fire- and smoke-related incidents.  Similar to renewal data, we will compute the aggregate values and percentage changes for the timespan of interest.

Note: The column "Total Exposure" seems to contain problematic data. The value for total exposure--the amount covered by the insurers--is much smaller than premiums they received in a particular zipcode, which doesn't make sense. Therefore, we're removing the data.

In [26]:
premiums = pd.read_excel(RAW_DATA_DIR / 'Residential-Property-Coverage-Amounts-Wildfire-Risk-and-Losses.xlsx', sheet_name='Cleaned', header=3)

# removing "Grand total" and "County" rows from the dataset 
premiums = premiums[premiums.Zipcode.apply(type) == int]

# getting columns for claims (n.) and losses ($)
cols_claim = []
cols_losses = []

for col in premiums.columns[6:]:
    if col[-6:] == 'Claims':
        cols_claim.append(col)
    else:
        cols_losses.append(col)

# calculating total numbers of claims and values in losses
premiums['Claims (Fire and Smoke)'] = premiums[cols_claim].sum(axis=1)
premiums['Losses (Fire and Smoke) ($)'] = premiums[cols_losses].sum(axis=1)

# keeping only the columns of interest
cols = ['Zipcode', 'Year', 'Earned Premium', 'Claims (Fire and Smoke)', 'Losses (Fire and Smoke) ($)']

# filtering and renaming columns
premiums = premiums[cols].rename(columns={'Zipcode': 'ZIP Code', 'Earned Premium': 'Earned Premium ($)'})
premiums['ZIP Code'] = premiums['ZIP Code'].astype(str)
premiums.sample(3)

,ZIP Code,Year,Earned Premium ($),Claims (Fire and Smoke),Losses (Fire and Smoke) ($)
20242,95365,2019,5092,0,0.0
30178,95932,2020,1453837,2,10226.0
54578,93256,2022,93170,0,0.0


In [27]:
# calculating the aggregate values for the timespan
cond1 = premiums['Year'] >= start_year
cond2 = premiums['Year'] <= end_year

premiums_aggs = premiums[cond1 & cond2].groupby(['ZIP Code']).sum().reset_index()
premiums_aggs = premiums_aggs.drop(columns=['Year'])

premiums_aggs.sample(3)

,ZIP Code,Earned Premium ($),Claims (Fire and Smoke),Losses (Fire and Smoke) ($)
1814,95206,43504133,208,13465267.0
876,92651,89710155,75,5906514.0
1988,95450,2015825,8,461715.0


In [56]:
# filtering years and getting growth
cond1 = premiums['Year'] == start_year
cond2 = premiums['Year'] == end_year

# creating pivot table with start and end years
premiums_pivot = pd.pivot_table(premiums[cond1 | cond2], index='ZIP Code', columns='Year').dropna()
premiums_pivot.columns = [f'{str(s[1])}_{s[0]}' for s in premiums_pivot.columns]
premiums_pivot = premiums_pivot.reset_index()

# calculating growth
premiums_pivot['% Change - Earned Premiums'] = premiums_pivot[['2018_Earned Premium ($)', '2021_Earned Premium ($)']].pct_change(axis=1).iloc[:, 1]
premiums_pivot['% Change - Claims (Fire and Smoke)'] = premiums_pivot[['2018_Claims (Fire and Smoke)', '2021_Claims (Fire and Smoke)']].pct_change(axis=1).iloc[:, 1]
premiums_pivot['% Change - Losses (Fire and Smoke)'] = premiums_pivot[['2018_Losses (Fire and Smoke) ($)', '2021_Losses (Fire and Smoke) ($)']].pct_change(axis=1).iloc[:, 1]

premiums_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
premiums_pivot.sample(3)

,ZIP Code,2018_Claims (Fire and Smoke),2021_Claims (Fire and Smoke),2018_Earned Premium ($),2021_Earned Premium ($),2018_Losses (Fire and Smoke) ($),2021_Losses (Fire and Smoke) ($),% Change - Earned Premiums,% Change - Claims (Fire and Smoke),% Change - Losses (Fire and Smoke)
1019,93561,2.857143,2.000000,1.363278e+06,1.957154e+06,75602.428571,204624.714286,0.435624,-0.300000,1.706589
1519,95252,4.142857,3.000000,6.750590e+05,1.084716e+06,240599.714286,226807.000000,0.606846,-0.275862,-0.057326
1609,95410,0.000000,0.166667,6.685029e+04,1.213792e+05,0.000000,40078.500000,0.815687,NaN,NaN


In [61]:
df1 = premiums_pivot[['ZIP Code', '% Change - Earned Premiums', '% Change - Claims (Fire and Smoke)', '% Change - Losses (Fire and Smoke)']].copy()
df2 = premiums_aggs.copy()

premiums_reworked = pd.merge(df2, df1, on='ZIP Code')
premiums_reworked.set_index('ZIP Code', inplace=True)

premiums_reworked.sample(3)

,Earned Premium ($),Claims (Fire and Smoke),Losses (Fire and Smoke) ($),% Change - Earned Premiums,% Change - Claims (Fire and Smoke),% Change - Losses (Fire and Smoke)
ZIP Code,,,,,,
93238,3694784,6,214092.0,0.622554,3.666667,70.064952
93111,25049840,32,2336207.0,0.320794,-0.636364,-0.479788
93954,176644,1,1141.0,0.641126,NaN,NaN


In [60]:
# TODO: move this one to feature engineering section

# # ratio between losses and premium
# premiums_reworked['ratio_losses_to_premium'] = premiums_reworked['fire_smoke_losses'] / premiums_reworked['earned_premium']

In [77]:
premiums_reworked.info(), premiums_reworked.index

<class 'pandas.core.frame.DataFrame'>
Index: 2082 entries, 90001 to 96162
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Earned Premium ($)                  2082 non-null   int64  
 1   Claims (Fire and Smoke)             2082 non-null   int64  
 2   Losses (Fire and Smoke) ($)         2082 non-null   float64
 3   % Change - Earned Premiums          2079 non-null   float64
 4   % Change - Claims (Fire and Smoke)  1406 non-null   float64
 5   % Change - Losses (Fire and Smoke)  1399 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 113.9+ KB


(None,
 Index(['90001', '90002', '90003', '90004', '90005', '90006', '90007', '90008',
        '90009', '90010',
        ...
        '96146', '96148', '96150', '96151', '96155', '96156', '96158', '96160',
        '96161', '96162'],
       dtype='object', name='ZIP Code', length=2082))

In [ ]:
premiums_reworked.to_csv(DATA_DIR / '2018_2021' / 'premiums_claims.csv')

readme = """
Dataset: premiums_claims.csv
Coverage: 2018-2021

'ZIP Code'
'earned_premium': total earned premium for the period
'earned_exposure': total exposure from the period * data doesn't seem to be consistent
'fire_smoke_claims': number of claims related to fire and smoke
'fire_smoke_losses': total amount ($) of losses related to fire and smoke
'growth_exposure': percentual change in the exposure between 2018 and 2021 * data doesn't seem to be consistent
'growth_premium': percentual change in earned premium between 2018 and 2021
'growth_fire_smoke_claims': percentual change in the number of claims related to fire and smoke
'growth_fire_smoke_losses': percentual change in the amounf of losses ($) related to fire and smoke
'ratio_losses_to_premium': ratio between losses and premium
"""

with open(DATA_DIR / '2018_2021' / 'premiums_claims.txt', 'w') as file:
    file.write(readme)

### <a id='toc1_2_3_'></a>[FAIR Plan (2022)](#toc0_)

In [92]:
fair22 = pd.read_excel(RAW_DATA_DIR / 'full_residential_units_insured_2022.xlsx')

cols = ['ZIP Code', "Voluntary Market Units", "FAIR Plan Units"]
fair22 = fair22[cols]

# calculate percentages
fair22['Total Res Units'] = fair22['Voluntary Market Units'] + fair22['FAIR Plan Units']
fair22['% Market Units'] = fair22['Voluntary Market Units'] / fair22['Total Res Units']
fair22['% FAIR Plan Units'] = fair22['FAIR Plan Units'] / fair22['Total Res Units']

fair22.head(2)

,ZIP Code,Voluntary Market Units,FAIR Plan Units,Total Res Units,% Market Units,% FAIR Plan Units
0,90001,6913,2104,9017,0.766663,0.233337
1,90002,6534,1330,7864,0.830875,0.169125


Besides the 2022 dataset, California also published general information about total exposure covered by FAIR Plan policies, which we're incorporating below. It's potentially a secondary target variable.

In [131]:
columns_exp = ['ZIP Code', 
               'growth_exp_23_24', 'exposure_24',
               'growth_exp_22_23', 'exposure_23',
               'growth_exp_21_22', 'Total Exposure ($)',
               'growth_exp_20_21', 'exposure_21',
               'exposure_20']

fair_exp = pd.read_excel(RAW_DATA_DIR / 'CFP5yearTIVGrowthbyzipcodethrough09302024(Residentialline)20241112v001_unlocked.xlsx', names=columns_exp)

# removing rows that doesn't contain actual data (totals, etc.)
from pandas.api.types import is_integer, is_number
fair_exp = fair_exp[fair_exp['ZIP Code'].apply(is_integer)].copy()

# cleaning up exposure data
fair_exp['Total Exposure ($)'] = fair_exp['Total Exposure ($)'].apply(lambda x: x if isinstance(x, int) else np.nan)

fair_exp.sample(3)

,ZIP Code,growth_exp_23_24,exposure_24,growth_exp_22_23,exposure_23,growth_exp_21_22,Total Exposure ($),growth_exp_20_21,exposure_21,exposure_20
621,91702,0.639,288577040,0.313,176097217,0.05,134124459.0,0.086,127739477,117651571
385,90013,1.197,5012334,0.331,2281620,-0.43,1714013.0,1.446,3004687,1228574
732,95460,1.185,122044948,0.435,55859394,0.547,38914143.0,0.768,25149876,14224192


In [132]:
# merging exposure column
fair = pd.merge(fair22, fair_exp[['ZIP Code', 'Total Exposure ($)']], on='ZIP Code')

In [133]:
fair.sample(3)

,ZIP Code,Voluntary Market Units,FAIR Plan Units,Total Res Units,% Market Units,% FAIR Plan Units,Total Exposure ($)
121,90638,11870,64,11934,0.994637,0.005363,33566972.0
127,90703,12978,48,13026,0.996315,0.003685,24368579.0
1137,95203,3388,67,3455,0.980608,0.019392,13651298.0


In [135]:
fair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ZIP Code                1631 non-null   object 
 1   Voluntary Market Units  1631 non-null   int64  
 2   FAIR Plan Units         1631 non-null   int64  
 3   Total Res Units         1631 non-null   int64  
 4   % Market Units          1631 non-null   float64
 5   % FAIR Plan Units       1631 non-null   float64
 6   Total Exposure ($)      1545 non-null   float64
dtypes: float64(3), int64(3), object(1)
memory usage: 89.3+ KB


In [ ]:
readme = """
Dataset: fair_plan2022.csv
Coverage: 2022

'ZIP Code'
'market_units': number of residential units covered by voluntary market insurance policies
'fair_plan_units': number of residential units covered by FAIR Plain policies
'total_units': total number of residential units covered by either option
'pct_market': percentage of units covered by market policies
'pct_fair_plan': percentage of units covered by FAIR Plan policies
'exposure_22': total exposure in FAIR Plan policies in USD
"""

with open(DATA_DIR / '2018_2021' / 'fair_plan2022.txt', 'w') as file:
    file.write(readme)

## <a id='toc1_3_'></a>[TODO: Zillow Data](#toc0_)

## <a id='toc1_4_'></a>[TODO: Disaster Data](#toc0_)

# <a id='toc2_'></a>[OLD CODE](#toc0_)

## <a id='toc2_1_'></a>[FAIR Plan 2 (2020-2024)](#toc0_)

This dataset contains FAIR Plan information for multiple years (2020-24) as well as information about the total exposure. However, it doesn't include data about the total market policies like the previous dataset.

Also, there are data for residential and commercial policies, but this notebook only deals with residential ones.

In [ ]:
# Policies
columns_pol = ['ZIP Code', 
               'growth_pol_23_24', 'policies_24',
               'growth_pol_22_23', 'policies_23',
               'growth_pol_21_22', 'policies_22',
               'growth_pol_20_21', 'policies_21',
               'policies_20']
fair2_pol = pd.read_excel(RAW_DATA_DIR / 'CFP5yearPIFGrowthbyzipcodethrough09302024(Residential+line)20241112v001_unlocked.xlsx', names=columns_pol)

# Exposure
columns_exp = ['ZIP Code', 
               'growth_exp_23_24', 'exposure_24',
               'growth_exp_22_23', 'exposure_23',
               'growth_exp_21_22', 'exposure_22',
               'growth_exp_20_21', 'exposure_21',
               'exposure_20']
fair2_exp = pd.read_excel(RAW_DATA_DIR / 'CFP5yearTIVGrowthbyzipcodethrough09302024(Residentialline)20241112v001_unlocked.xlsx', names=columns_exp)

In [81]:
# removing rows that doesn't contain actual data (totals, etc.)
from pandas.api.types import is_integer, is_number

fair2_pol = fair2_pol[fair2_pol['ZIP Code'].apply(is_integer)].copy()
fair2_exp = fair2_exp[fair2_exp['ZIP Code'].apply(is_integer)].copy()

fair2_pol.shape, fair2_exp.shape

((1647, 10), (1647, 10))

In [82]:
fair2_exp.head(3)

,ZIP Code,growth_exp_23_24,exposure_24,growth_exp_22_23,exposure_23,growth_exp_21_22,exposure_22,growth_exp_20_21,exposure_21,exposure_20
2,94501,0.676,98431342,0.179,58719416,0.137,49797731,0.091,43791971,40143917
3,94502,13.274,6880050,0.025,481983,1.85,470279,0,165000,165000
4,94536,1.995,40642190,0.852,13571624,0.307,7327808,0.264,5605823,4435256


In [83]:
fair2_pol.head(3)

,ZIP Code,growth_pol_23_24,policies_24,growth_pol_22_23,policies_23,growth_pol_21_22,policies_22,growth_pol_20_21,policies_21,policies_20
2,94501,0.333,104,0.04,78,-0.063,75,0,80,80
3,94502,2.5,7,0,2,1,2,0,1,1
4,94536,2.105,59,0.727,19,0.222,11,-0.1,9,10


In [84]:
# merging datasets
fair2 = pd.merge(fair2_pol, fair2_exp, on='ZIP Code')
fair2.head()

,ZIP Code,growth_pol_23_24,policies_24,growth_pol_22_23,policies_23,growth_pol_21_22,policies_22,growth_pol_20_21,policies_21,policies_20,growth_exp_23_24,exposure_24,growth_exp_22_23,exposure_23,growth_exp_21_22,exposure_22,growth_exp_20_21,exposure_21,exposure_20
0,94501,0.333,104,0.04,78,-0.063,75,0,80,80,0.676,98431342,0.179,58719416,0.137,49797731,0.091,43791971,40143917
1,94502,2.5,7,0,2,1,2,0,1,1,13.274,6880050,0.025,481983,1.85,470279,0,165000,165000
2,94536,2.105,59,0.727,19,0.222,11,-0.1,9,10,1.995,40642190,0.852,13571624,0.307,7327808,0.264,5605823,4435256
3,94538,1.4,24,0.667,10,0.2,6,0.25,5,4,1.445,15574256,1.647,6370385,0.771,2406677,0.352,1358996,1004964
4,94539,2.471,59,1.125,17,1.667,8,-0.25,3,4,1.934,79814473,0.983,27207162,2.696,13722261,-0.094,3712311,4096084


In [85]:
fair2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ZIP Code          1649 non-null   object
 1   growth_pol_23_24  1649 non-null   object
 2   policies_24       1649 non-null   object
 3   growth_pol_22_23  1649 non-null   object
 4   policies_23       1649 non-null   object
 5   growth_pol_21_22  1649 non-null   object
 6   policies_22       1649 non-null   object
 7   growth_pol_20_21  1649 non-null   object
 8   policies_21       1649 non-null   object
 9   policies_20       1649 non-null   object
 10  growth_exp_23_24  1649 non-null   object
 11  exposure_24       1649 non-null   object
 12  growth_exp_22_23  1649 non-null   object
 13  exposure_23       1649 non-null   object
 14  growth_exp_21_22  1649 non-null   object
 15  exposure_22       1649 non-null   object
 16  growth_exp_20_21  1649 non-null   object
 17  exposure_21   

In [86]:
# convert all columns to float and set to 0 empty values
def clean_non_ints(val):
    return 0 if is_number(val) == False else val

fair2 = fair2.map(clean_non_ints)


In [ ]:
fair2.to_csv(DATA_DIR / 'fair_plan.csv')

## <a id='toc2_2_'></a>[FEMA Projected Premium Increases (2021, 2025)](#toc0_)

FEMA created a methodology to predict monthly prices increases (Risk Rating 2.0) and publishes zipcode-level data based on it. The data consists of $10-increment columns with number of policies that they predict to change. They also have data for all the policies and for only single-housing units.

https://www.fema.gov/flood-insurance/risk-rating/profiles

In [ ]:
fema = pd.read_excel(RAW_DATA_DIR / 'fema_risk-rating-zip-breakdown-california_2021.xlsx', header=3, sheet_name='SFH Zip Count')

# drop State column and Grand Total row
fema.drop(columns='State', inplace=True)
fema = fema[fema.columns[:-1]]

fema.tail()

,Zip Code,< -$100,$-100 to $-90,$-90 to $-80,$-80 to $-70,$-70 to $-60,$-60 to $-50,$-50 to $-40,$-40 to $-30,$-30 to $-20,...,$10 to $20,$20 to $30,$30 to $40,$40 to $50,$50 to $60,$60 to $70,$70 to $80,$80 to $90,$90 to $100,> $100
1456,96161,6.0,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1457,CA Total of ZIPs w/ <5 Policies,43.0,4.0,4.0,5.0,4.0,7.0,4.0,5.0,5.0,...,29.0,7.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1458,CA Unknown ZIP,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,00052 <5 Policies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,NaN,7464.0,814.0,934.0,1277.0,1519.0,1835.0,1977.0,2094.0,2372.0,...,10326.0,3132.0,611.0,109.0,46.0,16.0,1.0,7.0,3.0,8.0


In [104]:
# removing totals and zipcodes with less than 5 policies and other non-zipcode-level data
fema = fema[fema['Zip Code'].str.len() == 5]

# replace NaN for 0 and force int (instead of float)
fema[fema.columns[1:]] = fema[fema.columns[1:]].replace(np.nan, 0).apply(pd.to_numeric).astype(int)

fema

,Zip Code,< -$100,$-100 to $-90,$-90 to $-80,$-80 to $-70,$-70 to $-60,$-60 to $-50,$-50 to $-40,$-40 to $-30,$-30 to $-20,...,$10 to $20,$20 to $30,$30 to $40,$40 to $50,$50 to $60,$60 to $70,$70 to $80,$80 to $90,$90 to $100,> $100
2,90004,1,0,0,0,2,0,0,0,1,...,2,0,0,0,0,0,0,0,0,0
3,90005,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
4,90006,0,0,0,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0
5,90007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,90008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,96145,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1453,96146,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1454,96148,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1455,96150,10,2,0,1,1,0,1,1,1,...,8,7,0,0,0,0,0,0,0,0


The original data is binned in very small buckets ($10 changes), which makes the information very hard to digest. I'll simplify to only 6 buckets and multiply for 12 months, to get the annual change.

In [105]:
# creating the bins
bin50_100_minus = fema.columns[2:7]
bin0_50_minus = fema.columns[7:12]
bin0_50 = fema.columns[12:17]
bin50_100 = fema.columns[17:22]


new_df = {'ZIP Code': fema['Zip Code'],
          '< -$100': fema['< -$100'],
          '-$100 to -$50': np.sum(fema[bin50_100_minus], axis=1),
          '-$50 to -$0': np.sum(fema[bin0_50_minus], axis=1),
          '$0 to $50': np.sum(fema[bin0_50], axis=1),
          '$50 to $100': np.sum(fema[bin50_100], axis=1),
          '> $100': fema['> $100']
}

projs = pd.DataFrame(new_df)
projs.sample(5)

,ZIP Code,< -$100,-$100 to -$50,-$50 to -$0,$0 to $50,$50 to $100,> $100
1310,95825,6,6,371,434,0,0
612,93221,3,2,5,20,0,0
110,90621,0,0,0,29,0,0
1115,95370,2,0,1,31,0,0
1377,96001,11,9,17,195,0,0


In [106]:
projs['n_decrease'] = np.sum(projs[projs.columns[1:4]], axis=1)
projs['n_increase'] = np.sum(projs[projs.columns[4:7]], axis=1)
projs['ratio_inc_to_dec'] = projs['n_decrease'] / projs['n_increase']
projs.sample(3)

,ZIP Code,< -$100,-$100 to -$50,-$50 to -$0,$0 to $50,$50 to $100,> $100,n_decrease,n_increase,ratio_inc_to_dec
1204,95565,1,0,2,3,0,0,3,3,1.000000
752,93657,13,23,24,111,0,0,60,111,0.540541
97,90405,0,0,0,55,0,0,0,55,0.000000


In [107]:
projs.to_csv(DATA_DIR / 'premium_change2021.csv')